In [ ]:
import zipfile
zip_file = zipfile.ZipFile('word2vec-nlp-tutorial.zip', 'r')
zip_file.extractall()
zip_file.close()

In [ ]:
/data/share/text7_ml_tm

In [1]:
import pandas as pd

# Файл labeledTrainData.tsv содержит тексты, которые размеченны по классам
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter="\t", quoting=3)

# Файл testData.tsv содержит тексты, по которым нужно выдать предсказания
test = pd.read_csv('testData.tsv', header=0,  delimiter="\t", quoting=3)

# Обратим внимание на unlabeledTrainData.tsv
# Для данного файла нет меток, к какому классу относятся его тексты
# Так же организаторы не ждут предсказания для по классам для текстов из данного файла
# В нем представленны тексты того же посола, что и остальные
# А значит добавив его в обучение word2vec мы улучшим знание нашемй модели о "мире"
unsup = pd.read_csv('unlabeledTrainData.tsv', header=0,  delimiter="\t", quoting=3)

Напишем функцию которая для представленного текста:
* меняет n't на not (по желанию)
* приводит к нижнему регистру
* делит на слова
* удаляет стоп-слова (по желанию)

In [3]:
from bs4 import BeautifulSoup
import re
#from nltk.corpus import stopwords

def text_to_wordlist(text):
    text = re.sub('n\'t', ' not', text)
    
    text = re.sub('[^a-zA-Z]', ' ', text)
    words = text.lower().split()

    #stops = set(stopwords.words("english"))

    return words

In [4]:
# Протестируем данную функцию

text_to_wordlist("""
Tyrone Garland <s>(born 1992)</s> is an American professional basketball player 
who last played with the National <b>Basketball</b> League of Canada's Mississauga Power
""")

# Посмотрим что результатом будет list, элементы которого - слова

['tyrone',
 'garland',
 's',
 'born',
 's',
 'is',
 'an',
 'american',
 'professional',
 'basketball',
 'player',
 'who',
 'last',
 'played',
 'with',
 'the',
 'national',
 'b',
 'basketball',
 'b',
 'league',
 'of',
 'canada',
 's',
 'mississauga',
 'power']

Напишем функцию которая для представленного текста:
* удаляет html теги
* производит деления на предложения
* каждое предложение делит на слова (применяя выше написанную функцию)

In [5]:
import nltk.data

def text_to_sentences(text):
    text = BeautifulSoup(text).get_text()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_to_wordlist(raw_sentence))

    return sentences

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Протестируем данную функцию

text_to_sentences("""
Sherlock Holmes is a four-act play written by <p/> William Gillette and Sir Arthur Conan Doyle, 
based on Conan Doyle's eponymous character. It drew material from the stories 
<s>"A Scandal in Bohemia"</s>, "The Final Problem", and A Study in Scarlet, pitting Holmes 
against Professor Moriarty and reinventing the character of Irene Adler as a new love 
interest named Alice Faulkner. This play introduced the phrase "Elementary, my dear Watson" 
and Holmes' curved pipe.
""")


# Посмотрим что результатом будет list, элементы которого - list, элементы которого - слова
# т.е. получаем list list'ов (список списков)

[['sherlock',
  'holmes',
  'is',
  'a',
  'four',
  'act',
  'play',
  'written',
  'by',
  'william',
  'gillette',
  'and',
  'sir',
  'arthur',
  'conan',
  'doyle',
  'based',
  'on',
  'conan',
  'doyle',
  's',
  'eponymous',
  'character'],
 ['it',
  'drew',
  'material',
  'from',
  'the',
  'stories',
  'a',
  'scandal',
  'in',
  'bohemia',
  'the',
  'final',
  'problem',
  'and',
  'a',
  'study',
  'in',
  'scarlet',
  'pitting',
  'holmes',
  'against',
  'professor',
  'moriarty',
  'and',
  'reinventing',
  'the',
  'character',
  'of',
  'irene',
  'adler',
  'as',
  'a',
  'new',
  'love',
  'interest',
  'named',
  'alice',
  'faulkner'],
 ['this',
  'play',
  'introduced',
  'the',
  'phrase',
  'elementary',
  'my',
  'dear',
  'watson',
  'and',
  'holmes',
  'curved',
  'pipe']]

Так давайте соберем все имеющиеся тексты в подобню структуру

In [9]:
%%time
sentences = []

for review in train["review"]:
    sentences += text_to_sentences(review)

CPU times: user 36.5 s, sys: 1.29 s, total: 37.8 s
Wall time: 38 s


In [10]:
%%time
for review in unsup["review"]:
    sentences += text_to_sentences(review)

CPU times: user 1min 14s, sys: 2.67 s, total: 1min 17s
Wall time: 1min 18s


In [11]:
%%time
for review in test["review"]:
    sentences += text_to_sentences(review)

CPU times: user 37.4 s, sys: 1.33 s, total: 38.7 s
Wall time: 39.1 s


In [12]:
# В конце сбора sentences будет list list'ов (список списков) - как и пример выше.
# (Повторюсь) каждый элемент списка sentences - предложение, но представленное в виде списка слов - потому список

# выведем количество элементов этого массива (оно же - количество предложений во всех текста)
print(len(sentences))

1059231


In [13]:
# а так же посмотрим на сам массив
print ('\t Первый элемент массива')
print (sentences[0]) 
print ('\t Второй элемент массива')
print (sentences[1])

	 Первый элемент массива
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
	 Второй элемент массива
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


Обучим же теперь модель Word2Vec

In [14]:
import logging 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
log = logging.getLogger()
log.setLevel(logging.INFO)

In [15]:
%%time
from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
params = {}
params['size'] = 300  # итоговая размерность вектора каждого слова
params['min_count'] = 5  # минимальная частотность слова, чтобы оно попало в модель
params['workers'] = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
params['window'] = 10        # размер окна 
params['sample'] = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, **params)

2019-04-27 13:19:53,417 : INFO : 'pattern' package not found; tag filters are not available for English
2019-04-27 13:19:53,424 : INFO : collecting all words and their counts
2019-04-27 13:19:53,425 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-27 13:19:53,519 : INFO : PROGRESS: at sentence #10000, processed 225522 words, keeping 17764 word types
2019-04-27 13:19:53,606 : INFO : PROGRESS: at sentence #20000, processed 451018 words, keeping 24916 word types
2019-04-27 13:19:53,702 : INFO : PROGRESS: at sentence #30000, processed 669878 words, keeping 30005 word types
2019-04-27 13:19:53,803 : INFO : PROGRESS: at sentence #40000, processed 895595 words, keeping 34309 word types
2019-04-27 13:19:53,879 : INFO : PROGRESS: at sentence #50000, processed 1114855 words, keeping 37727 word types
2019-04-27 13:19:53,939 : INFO : PROGRESS: at sentence #60000, processed 1335610 words, keeping 40685 word types
2019-04-27 13:19:54,009 : INFO : PROGRESS: at senten

2019-04-27 13:20:00,130 : INFO : PROGRESS: at sentence #710000, processed 15844652 words, keeping 117479 word types
2019-04-27 13:20:00,237 : INFO : PROGRESS: at sentence #720000, processed 16071484 words, keeping 118118 word types
2019-04-27 13:20:00,358 : INFO : PROGRESS: at sentence #730000, processed 16294148 words, keeping 118833 word types
2019-04-27 13:20:00,430 : INFO : PROGRESS: at sentence #740000, processed 16516151 words, keeping 119544 word types
2019-04-27 13:20:00,526 : INFO : PROGRESS: at sentence #750000, processed 16735891 words, keeping 120182 word types
2019-04-27 13:20:00,590 : INFO : PROGRESS: at sentence #760000, processed 16953895 words, keeping 120821 word types
2019-04-27 13:20:00,657 : INFO : PROGRESS: at sentence #770000, processed 17179782 words, keeping 121573 word types
2019-04-27 13:20:00,738 : INFO : PROGRESS: at sentence #780000, processed 17408560 words, keeping 122302 word types
2019-04-27 13:20:00,826 : INFO : PROGRESS: at sentence #790000, processe

2019-04-27 13:20:32,611 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-04-27 13:20:32,616 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-04-27 13:20:32,631 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-04-27 13:20:32,633 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-04-27 13:20:32,635 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-27 13:20:32,651 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-27 13:20:32,676 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-27 13:20:32,678 : INFO : EPOCH - 1 : training on 23584060 raw words (17473737 effective words) took 27.4s, 637842 effective words/s
2019-04-27 13:20:33,725 : INFO : EPOCH 2 - PROGRESS: at 2.88% examples, 499143 words/s, in_qsize 15, out_qsize 0
2019-04-27 13:20:34,737 : INFO : EPOCH 2 - PROGRESS: at 6.10% examples, 527196 words/s, in_qsize 15, out_qsize

2019-04-27 13:21:26,411 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-27 13:21:26,412 : INFO : EPOCH - 3 : training on 23584060 raw words (17474709 effective words) took 27.1s, 644015 effective words/s
2019-04-27 13:21:27,482 : INFO : EPOCH 4 - PROGRESS: at 3.25% examples, 545707 words/s, in_qsize 15, out_qsize 0
2019-04-27 13:21:28,488 : INFO : EPOCH 4 - PROGRESS: at 6.86% examples, 584354 words/s, in_qsize 15, out_qsize 0
2019-04-27 13:21:29,490 : INFO : EPOCH 4 - PROGRESS: at 10.50% examples, 598734 words/s, in_qsize 15, out_qsize 0
2019-04-27 13:21:30,501 : INFO : EPOCH 4 - PROGRESS: at 13.99% examples, 598905 words/s, in_qsize 15, out_qsize 0
2019-04-27 13:21:31,531 : INFO : EPOCH 4 - PROGRESS: at 17.31% examples, 591017 words/s, in_qsize 14, out_qsize 1
2019-04-27 13:21:32,532 : INFO : EPOCH 4 - PROGRESS: at 20.18% examples, 576528 words/s, in_qsize 15, out_qsize 0
2019-04-27 13:21:33,541 : INFO : EPOCH 4 - PROGRESS: at 23.98% examples, 587249 words/s

2019-04-27 13:22:25,706 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-27 13:22:25,707 : INFO : EPOCH - 5 : training on 23584060 raw words (17473130 effective words) took 29.4s, 594919 effective words/s
2019-04-27 13:22:25,708 : INFO : training on a 117920300 raw words (87369144 effective words) took 140.4s, 622140 effective words/s


CPU times: user 9min 57s, sys: 6 s, total: 10min 3s
Wall time: 2min 32s


In [16]:
# Финализируем нашу модель. Ее нельзя будет доучить теперь, но она начнет занимать гораздо меньше места
model.init_sims(replace=True)

2019-04-27 13:26:50,507 : INFO : precomputing L2-norms of word weight vectors


Натренировав модель, получили представление каждого слова в семантическом пространстве (часто называют "псевдо" семантическое пространство)

Попробуем популярный пример: QUEEN + MAN - KING = ? 

In [20]:
model.most_similar(positive=['queen', 'man'], negative=['king'])

[('hawking', 0.39587709307670593),
 ('men', 0.36946797370910645),
 ('himself', 0.36450180411338806),
 ('guy', 0.3641517460346222),
 ('filmmaker', 0.3599601984024048),
 ('nemesis', 0.3579413890838623),
 ('soldier', 0.3561764657497406),
 ('alvin', 0.3560636639595032),
 ('lad', 0.3560309410095215),
 ('donner', 0.35371023416519165)]

Получилось WOMAN, что логично :) 

In [21]:
# Посмотрим слова, которые очень похожи на слово MOVIE

model.most_similar('movie')

# Попробуем и другие слова

[('film', 0.854441225528717),
 ('flick', 0.6861598491668701),
 ('movies', 0.5816400647163391),
 ('it', 0.5664041638374329),
 ('sequel', 0.5207743644714355),
 ('picture', 0.513239860534668),
 ('miniseries', 0.4933973550796509),
 ('documentary', 0.49095818400382996),
 ('show', 0.483186811208725),
 ('stinker', 0.478931725025177)]

In [22]:
model.most_similar('awful')

[('terrible', 0.7712875008583069),
 ('atrocious', 0.7404935359954834),
 ('horrible', 0.7230589389801025),
 ('dreadful', 0.6988893747329712),
 ('abysmal', 0.6940825581550598),
 ('appalling', 0.6884695291519165),
 ('horrid', 0.6619865894317627),
 ('horrendous', 0.650810956954956),
 ('lousy', 0.6421493291854858),
 ('amateurish', 0.6044957637786865)]

In [23]:
model.most_similar('big')

[('huge', 0.6679580807685852),
 ('major', 0.4751586318016052),
 ('biggest', 0.4730166792869568),
 ('mega', 0.4682440757751465),
 ('massive', 0.43350180983543396),
 ('silver', 0.4298832416534424),
 ('bigger', 0.4158284664154053),
 ('miniscule', 0.4135412573814392),
 ('minuscule', 0.4091233015060425),
 ('savers', 0.3983842134475708)]

In [24]:
model.most_similar('mail')

[('email', 0.6582958698272705),
 ('yahoo', 0.554697573184967),
 ('mails', 0.5379413366317749),
 ('html', 0.5329873561859131),
 ('hotmail', 0.5301001667976379),
 ('aol', 0.5283646583557129),
 ('download', 0.5272182822227478),
 ('matin', 0.5097979307174683),
 ('magazines', 0.501447856426239),
 ('letters', 0.49451926350593567)]

In [27]:
# так же взглянем на функцию doesnt_match, она покажет лишнее слово в массив

model.doesnt_match(['berlin', 'moscow', 'africa', 'rome', 'paris'])

'moscow'

In [30]:
model.doesnt_match(['man', 'woman', 'child', 'dog'])

'dog'

In [31]:
# давайте просмотрим вектор одного из слов
# его длину
print (len(model['moon']))

# и сам вектор
print (model['moon'])

300
[-2.41765045e-02  1.13930516e-02 -7.65231401e-02  1.34541765e-01
 -5.49982674e-02 -1.80851985e-02 -3.10797468e-02 -1.10409237e-01
  6.21234216e-02 -9.40666422e-02 -1.31945843e-02  3.60658616e-02
 -5.46340831e-02 -3.95423314e-03  3.73740755e-02  5.67800663e-02
 -1.43558607e-01  5.73062785e-02 -1.83172941e-01 -9.55514051e-03
 -2.78906915e-02  6.34366348e-02  6.81844279e-02 -8.58814269e-02
  9.13262460e-03  3.08009293e-02  2.22716555e-02 -6.25756159e-02
  1.69782974e-02 -1.98944025e-02  2.02695411e-02  1.42104356e-02
 -1.42527595e-02 -4.64156531e-02  8.75117034e-02  2.32919585e-02
 -2.09304038e-02 -3.89328972e-02 -2.72467895e-03 -8.74985158e-02
  6.51913062e-02  7.38173425e-02  3.87203507e-02  4.50330600e-03
  2.38494184e-02 -6.40513971e-02  3.16905566e-02 -5.52261584e-02
  9.94774103e-02 -3.16848904e-02 -8.82579312e-02 -5.37963361e-02
 -1.58704780e-02 -1.01641469e-01  5.25919488e-03  8.60715210e-02
  8.82308558e-02  1.27134249e-01  5.67219593e-02 -3.35409045e-02
  5.38940467e-02 -1.2

In [32]:
# Словарь - все слова которые участвуют в модели можно просмотреть так
len(model.wv.vocab.keys())

53172

Т.к. мы хотим классифицировать не слова, а тексты, надо перевести тексты в вектора (представить в виде фич)
Один из простых методов - сложить все вектора слов входящих в текст и поделить на число входящих слов.
Напишем функцию, которая:
* создает нулевой вектор - это будет результирующий вектор
* идем по всем словам в тексте, если слово есть в моделе:
  * увеличиваем счетчик слов
  * прибавим вектор слова к результирующему вектору
* поделим все координаты на число слов, вектора которых мы прибавляли к результирующему вектору

In [33]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.vocab.keys())
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

Напишем функцию, которая на входе получает список всех текстов, а на выходе отдает список вектор каждого текста - что является прямоугольной матрицей

In [34]:
from tqdm import tqdm_notebook as tqdm
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in tqdm(enumerate(texts),):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

Заметим что функция texts_to_vecs принимает не просто тексты, а список всех слов текста. 

(!!!) Внимание: не список списков (там где сначала делили на предложения, а предложения на слова), а обычный линейный список

Но у нас есть функции, которые переводят 1) текст в список предложений, 2) предложение в список слов

Может показаться, что можно использовать 2ую функцию, но придется тогда ее переписать, потому как теги у нас удаляются лишь в первой функции

In [35]:
# Поступим иначе, в python есть возможность развернуть двухмерный массив в одномерный, вот пример

temp_list = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]
print (sum(temp_list, []))

# магия :)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [36]:
# Мы же такой возможностью воспользуемся, зная что функция text_to_sentences возвращает список списков

list_of_list_of_words = text_to_sentences("""
Sherlock Holmes is a four-act play written by <p/> William Gillette and Sir Arthur Conan Doyle, 
based on Conan Doyle's eponymous character. It drew material from the stories 
<s>"A Scandal in Bohemia"</s>, "The Final Problem", and A Study in Scarlet, pitting Holmes 
against Professor Moriarty and reinventing the character of Irene Adler as a new love 
interest named Alice Faulkner. This play introduced the phrase "Elementary, my dear Watson" 
and Holmes' curved pipe.
""")

print ('\t длина list_of_list_of_words')
print (len(list_of_list_of_words))
print ('\t первый элемент list_of_list_of_words')
print (list_of_list_of_words[0])

list_of_words = sum(list_of_list_of_words, [])

print ('\t длина list_of_words')
print (len(list_of_words))
print ('\t первый элемент list_of_words')
print (list_of_words[0])

	 длина list_of_list_of_words
3
	 первый элемент list_of_list_of_words
['sherlock', 'holmes', 'is', 'a', 'four', 'act', 'play', 'written', 'by', 'william', 'gillette', 'and', 'sir', 'arthur', 'conan', 'doyle', 'based', 'on', 'conan', 'doyle', 's', 'eponymous', 'character']
	 длина list_of_words
74
	 первый элемент list_of_words
sherlock


In [37]:
# действительно работает, сделаем для всех текстов из train
from tqdm import tqdm_notebook as tqdm
train_like_word_list = [sum(text_to_sentences(text), []) for text in tqdm(train['review'], total=len(train['review']))]

In [38]:
train_vecs = texts_to_vecs(train_like_word_list, model, params['size'])

In [41]:
# сделаем тоже самое для test
test_like_word_list = [sum(text_to_sentences(text), []) for text in tqdm(test['review'])]

In [42]:
test_vecs = texts_to_vecs(test_like_word_list, model, params['size'])

In [44]:
%%time
# Воспользуемся train_vecs, test_vecs, train["sentiment"] 
#    как матрица фичей обучающей выборки, матрица фичей тестовой выборки, таргет для обучающей выборки соответственно
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_vecs,  train["sentiment"])
# Стандартный случайный лес в таком случае
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(X_train,y_train)
predict = forest.predict_proba(X_test)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, predict[:,1]))
# И вот задача решена

0.9130505555594812
CPU times: user 31 s, sys: 132 ms, total: 31.1 s
Wall time: 4.12 s


Но что если для получения результирующего вектора не складывать вектора, а пойти другим способом.

Кластеризуем все слова на 1000 классов.

In [ ]:
# В model.syn0 хранятся все вектора. Кластеризуем их!

print ('Размер ', model.wv.syn0.shape)
print ('Вектор ', model.wv.syn0[0])

In [40]:
%%time
# Кластеризируем все слова. 

from sklearn.cluster import KMeans

word_vectors = model.wv.syn0
# Число кластеров установим в 1000. Для этого числа нет "серебряной пули". Для каждого случая лучше подойдет разная
num_clusters = 1000

# Начнем кластеризацию, учитывая что классов много, количество векторов (по сути слов) много,
#   все это будет происходит продолжительное время. Можно сходить за чаем.
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# в idx будут храниться номера классов для каждого слова

KeyboardInterrupt: 

In [ ]:
# создадим структуру dict (словарь): слово -> класс
word_centroid_map = dict(zip(model.wv.index2word, idx))

In [ ]:
import pickle
pickle.dump(word_centroid_map,open('word_centroid_map.pkl','wb'))

In [ ]:
import pickle
word_centroid_map = pickle.load(open('word_centroid_map.pkl','rb'))

In [ ]:
word_centroid_map['rhyming']

In [ ]:
# выведем представителей первых 10 классов и посмотрим на адекватность произошедшей кластеризации
clust = list(word_centroid_map.values())
wrds = list(word_centroid_map.keys())
for cluster in range(0,10):
    print (cluster)
    words = []

    for i in range(0, len(clust)):
        if clust[i] == cluster:
            words.append(wrds[i])
    print (words)

Существует несколько подходов к работе с кластерами слов. Рассмотрим 2 примера
* 1) посчитаем для каждого текста вектор, сколько его слов встретилось в каждом кластере. Т.е. для каждого текста будет вектор размера, равного числу кластеров, значениями вектора будут натуральные числа.
* 2) Посчитаем усредненных удаленность векторов текстов от каждого центроида всех кластеров.

In [ ]:
# для первого случая напишем функцию, на вход которой поступает текст, представленный в виде списка всех его слов
#    смотрим в каком кластере находится каждое слово 
#    и увеличиваем соответствующую ячейку (ответственную за этот кластер) на 1


def create_bag_of_centroids(wordlist, word_centroid_map, num_centroids):
    bag_of_centroids = np.zeros(num_centroids, dtype="float32")
    set_word_centroid_map = set(word_centroid_map.keys())
    
    for word in wordlist:
        if word in set_word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    return bag_of_centroids

In [ ]:
%%time
# Но нам нужно это не для одного текста, а для всех текстов обучающей и тестовой выборки
# Сделаем это

train_vecs_centroids = np.zeros((train["review"].size, num_clusters), dtype="float32")

for i, text in enumerate(train_like_word_list):
    train_vecs_centroids[i] = create_bag_of_centroids(text, word_centroid_map, num_clusters)

test_vecs_centroids = np.zeros((test["review"].size, num_clusters), dtype="float32")

for i, text in enumerate(test_like_word_list):
    test_vecs_centroids[i] = create_bag_of_centroids(text, word_centroid_map, num_clusters)
    
# Результатом будет матрицы для обучающих и тестовых текстов, что будет матрицей фич обучающей и тестовой выборок соотв.

In [ ]:
# стандартный случайный лес на полученных матрицах

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs_centroids, train["sentiment"])
predict = forest.predict(test_vecs_centroids)

---

In [ ]:
%%time
# а теперь попробуем посчитать растояние от центроидов

train_vecs_centroids_dist = np.zeros((train["review"].size, num_clusters), dtype="float32")

from scipy.spatial import distance

for i, vec in tqdm(enumerate(train_vecs), total=len(train_vecs)):
    for j, center in enumerate(kmeans_clustering.cluster_centers_):
        train_vecs_centroids_dist[i][j] = distance.euclidean(vec, center)

In [ ]:
train_vecs_centroids_dist.shape

In [ ]:
test_vecs_centroids_dist = np.zeros((test["review"].size, num_clusters), dtype="float32")

for i, vec in tqdm(enumerate(test_vecs), total=len(test_vecs)):
    for j, center in enumerate(kmeans_clustering.cluster_centers_):
        test_vecs_centroids_dist[i][j] = distance.euclidean(vec, center)

In [ ]:
test_vecs_centroids_dist.shape

In [ ]:
# стандартный случайный лес на полученных матрицах (кто бы сомневался)

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs_centroids_dist, train["sentiment"])
predict = forest.predict(test_vecs_centroids_dist)